In [1]:
import re
import jsons
import time
import logging
import os
import requests
import speech_recognition as sr
from os import path
from pydub import AudioSegment
import pandas as pd
from csv import DictReader
import google.generativeai as genai
import subprocess
ffmpeg_path = r"C:\Users\Samiksha Shinde\Desktop\web scraping project\ffmpeg-7.1-full_build\bin\ffmpeg.exe"
from gec_comman import captcha_fill_func as cf
from selenium.webdriver.chrome.options import Options
from datetime import date, datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium import webdriver
from selenium.webdriver.common.by import By
from deep_translator import GoogleTranslator
from selenium.webdriver.support.ui import Select
notice_count = 0

def extract_and_save_notice(tender_html_element):
    global notice_count

    try: 
        notice_no = tender_html_element.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text
        print("notice_no::",notice_no)
    except Exception as e:
        logging.info("Exception in notice_no: {}".format(type(e).__name__))
        pass

    org_name = tender_html_element.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').text

    try:
        local_title = tender_html_element.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text
        notice_title = GoogleTranslator(source='es', target='en').translate(local_title)
    except Exception as e:
        logging.info("Exception in local_title: {}".format(type(e).__name__))
        pass

    try: 
        publish_date = tender_html_element.find_element(By.CSS_SELECTOR, 'td:nth-child(6) > div > span > span').text
        publish_date = re.findall('\d+/\d+/\d{4} \d+:\d+ \w+',publish_date)[0]
        publish_date = datetime.strptime(publish_date,'%d/%m/%Y %I:%M %p').strftime('%Y/%m/%d %H:%M:%S')
        print(publish_date)
    except Exception as e:
        logging.info("Exception in publish_date: {}".format(type(e).__name__))
        pass

    if publish_date is not None and publish_date < threshold:
        return

    try:
        est_amount = tender_html_element.find_element(By.CSS_SELECTOR, 'td:nth-child(8)').text
        est_amount = re.sub("[^\d\.\,]", "", est_amount) 
        est_amount = float(est_amount.replace('.','').replace(',','.').strip())
        netbudgetlc = est_amount
    except Exception as e:
        logging.info("Exception in est_amount : {}".format(type(e).__name__))
        pass

    try:
        document_type_description = tender_html_element.find_element(By.CSS_SELECTOR, 'td:nth-child(9)').text
    except Exception as e:
        logging.info("Exception in document_type_description : {}".format(type(e).__name__))
        pass
        
    try:
        notice_text += tender_html_element.get_attribute('outerHTML')
    except:
        pass

    try:
        notice_url = tender_html_element.find_element(By.CSS_SELECTOR,'td:nth-child(10) a').get_attribute('onclick').split("'noticeUID=' + '")[1].split("'")[0].strip()
        notice_url =  'https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID='+str(notice_url)
        print(notice_url)
        page_details.get(notice_url)
    except:
        pass

    try:
        cf.captcha_filling_fun(page_details,"#tblContainer > tbody")
        time.sleep(5)
        os.remove("output.wav")
        os.remove("captcha_audio.mp3")
    except:
        pass
    
    WebDriverWait(page_details, 120).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[2]/div/form/table/tbody/tr[4]/td/table/tbody/tr[1]/td/fieldset/div/table/tbody/tr[2]'))).text

    for i in range(1,3):
        i = page_details.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(5)

    try:
        local_description = page_details.find_element(By.XPATH,'''//*[contains(text(),"Descripción:")]//following::td[1]/span[1]''').text
        notice_summary_english = GoogleTranslator(source='es', target='en').translate(local_description)
    except Exception as e:
        logging.info("Exception in local_description: {}".format(type(e).__name__))
        pass

    try:
        contract_type_actual = page_details.find_element(By.XPATH,'''//*[contains(text(),"Tipo de contrato")]//following::td[1]/span[1]''').text
        if 'Prestación de servicios' in contract_type_actual:
            notice_contract_type = 'Service'
        elif 'Goods' in contract_type_actual:
            notice_contract_type = 'Supply'
        elif 'Works' in contract_type_actual:
            notice_contract_type = 'Works'
        else:
            pass
    except Exception as e:
        logging.info("Exception in notice_contract_type: {}".format(type(e).__name__))
        pass

    try:
        tender_contract_end_date = page_details.find_element(By.XPATH, '''//*[contains(text(),"Fecha de terminación del contrato:")]//following::td[1]''').text
        tender_contract_end_date = re.findall('\d+/\d+/\d{4} \d+:\d+:\d+ [ampmAMPM]+',tender_contract_end_date)[0]
        tender_contract_end_date = datetime.strptime(tender_contract_end_date,'%d/%m/%Y %H:%M:%S %p').strftime('%Y/%m/%d %H:%M:%S')
    except Exception as e:
        logging.info("Exception in tender_contract_end_date: {}".format(type(e).__name__))
        pass

    try:
        contract_duration = page_details.find_element(By.XPATH, '//*[contains(text(),"Duración del contrato:")]//following::td[1]').text
    except Exception as e:
        logging.info("Exception in contract_duration: {}".format(type(e).__name__))
        pass

    try:
        notice_text += page_details.find_element(By.XPATH, '//*[@id="tblContainer_trRowContent"]').get_attribute('outerHTML')
    except Exception as e:
        logging.info("Exception in notice_text: {}".format(type(e).__name__))
        pass
    
    try:              
        org_name = org_name
        
        try:
            org_address = page_details.find_element(By.XPATH, '//*[contains(text(),"Dirección de ejecución del contrato")]//following::td[1]').text
        except Exception as e:
            logging.info("Exception in org_address: {}".format(type(e).__name__))
            pass 
    except Exception as e:
        logging.info("Exception in customer_details: {}".format(type(e).__name__)) 
        pass

    try:
        cpv_at_source = ''
        codes = page_details.find_element(By.XPATH, '''//*[@id='fdsObjectOfTheContract_tblDetail_trRowMainObjectCPV_tdCellCat']''').text
        cpv_at_source_regex = re.compile(r'\d{8}')
        cpv_list = cpv_at_source_regex.findall(codes)
        for code in cpv_list:
            cpv_code = code

            cpv_at_source += code
            cpv_at_source += ','
    
        cpv_at_source = cpv_at_source.rstrip(',')
    except Exception as e:
        logging.info("Exception in cpv_code: {}".format(type(e).__name__))
        pass

    try:
        class_codes_at_source = ''
        codes_at_source = page_details.find_element(By.XPATH, '''//*[@id='fdsObjectOfTheContract_tblDetail_trRowMainObjectCPV_tdCellCat']''').text
        cpv_regex = re.compile(r'\d{8}')
        code_list = cpv_regex.findall(codes_at_source)
        for codes in code_list:
            class_codes_at_source += codes
            class_codes_at_source += ','
        class_codes_at_source = class_codes_at_source.rstrip(',')
    except Exception as e:
        logging.info("Exception in cpvs: {}".format(type(e).__name__)) 
        pass

    try:
        lot_number = 1
        for single_record in page_details.find_elements(By.CSS_SELECTOR, 'table.PriceListLineTable > tbody > tr.PriceListLineRow'):
            lot_number = lot_number

            try:
                lot_class_codes_at_source = ''
                lot_cpv_at_source = ''
                codes_at_source = single_record.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').text
                cpv_regex = re.compile(r'\d{8}')
                code_list = cpv_regex.findall(codes_at_source)
                for cpv_codes in code_list:
                        lot_cpvs_data.lot_cpv_code = cpv_codes
                        
                        lot_class_codes_at_source += cpv_codes
                        lot_class_codes_at_source += ','
                        
                        lot_cpv_at_source += cpv_codes
                        lot_cpv_at_source += ','
                        
                lot_class_codes_at_source = lot_class_codes_at_source.rstrip(',')
                lot_cpv_at_source = lot_cpv_at_source.rstrip(',')
            except:
                pass
    
            lot_title = single_record.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text
            lot_title_english = GoogleTranslator(source='auto', target='en').translate(lot_title)
    
            try:
                l_quantity = single_record.find_element(By.CSS_SELECTOR, 'td:nth-child(4)').text 
                lot_quantity1 = l_quantity.replace(',','.').strip()
                lot_quantity = float(lot_quantity1)
            except Exception as e:
                logging.info("Exception in lot_quantity: {}".format(type(e).__name__))
                pass

            try:
                lot_quantity_uom = single_record.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text
            except Exception as e:
                logging.info("Exception in lot_quantity_uom: {}".format(type(e).__name__))
                pass
    
            try:
                lot_netbudget_lc = single_record.find_element(By.CSS_SELECTOR, 'td:nth-child(6)').text
                lot_netbudget_lc = re.sub("[^\d\.]", "", lot_netbudget_lc) 
                lot_netbudget_lc = float(lot_netbudget_lc.replace('.','').strip()) 
            except Exception as e:
                logging.info("Exception in lot_netbudget_lc: {}".format(type(e).__name__))
                pass

            try:
                contract_end_date = page_details.find_element(By.XPATH, '''//*[contains(text(),"Fecha de terminación del contrato:")]//following::td[1]''').text
                contract_end_date = re.findall('\d+/\d+/\d{4} \d+:\d+:\d+ [ampmAMPM]+',contract_end_date)[0]
                contract_end_date = datetime.strptime(contract_end_date,'%d/%m/%Y %H:%M:%S %p').strftime('%Y/%m/%d %H:%M:%S')
            except Exception as e:
                logging.info("Exception in contract_end_date: {}".format(type(e).__name__))
                pass    

            lot_number += 1
    except Exception as e:
        logging.info("Exception in lot_details: {}".format(type(e).__name__)) 
        pass   

    notice_count += 1
    print('----------------------------------')
# ----------------------------------------- Main Body

options = Options()
# options.add_argument("--headless")
options.add_argument('--no-sandbox')
options.add_argument('--incognito')
options.add_argument('--disable-dev-shm-usage') 
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--ignore-gpu-blacklist')
options.add_argument("window-size=1024x768")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
                     
page_main= webdriver.Chrome(options=options)
page_details= webdriver.Chrome(options=options)
# urls = ["https://riebbs.ac.in/tenders.php"] 

try:
    th = date.today() - timedelta(1)
    threshold = th.strftime('%Y/%m/%d')
    urls = ["https://community.secop.gov.co/Public/Tendering/ContractNoticeManagement/Index?currentLanguage=es-CO&Page=login&Country=CO&SkinName=CCE"] 
    for url in urls:
        page_main.get(url)
        print(url)
        time.sleep(5) #captcha
        
        cf.captcha_filling_fun(page_main,"#selRequestStatus")
        time.sleep(5)
        os.remove("output.wav")
        os.remove("captcha_audio.mp3")
            
        select_btn = Select(page_main.find_element(By.CSS_SELECTOR,'#selRequestStatus'))
        select_btn.select_by_index(3)
        time.sleep(5)

        click_buscar = WebDriverWait(page_main, 50).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#btnSearchButton')))
        page_main.execute_script("arguments[0].click();",click_buscar)
        time.sleep(5)
        
        try:
            rows = WebDriverWait(page_main, 120).until(EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[2]/div[2]/table/tbody/tr/td[3]/form/table/tbody/tr[2]/td/table/tbody/tr[6]/td/div[1]/table/tbody/tr')))
            length = len(rows)
        except:
            logging.info('No new record')
            break   
        for records in range(1,length-1):
            tender_html_element = WebDriverWait(page_main,100).until(EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[2]/div[2]/table/tbody/tr/td[3]/form/table/tbody/tr[2]/td/table/tbody/tr[6]/td/div[1]/table/tbody/tr')))[records]
            extract_and_save_notice(tender_html_element)
                 
#         logging.info("Finished processing. Scraped {} notices".format(notice_count))
except Exception as e:
    raise e
    logging.info("Exception:"+str(e))
# finally:
#     page_main.quit()
#     page_details.quit()

<>:47: SyntaxWarning: invalid escape sequence '\d'
<>:59: SyntaxWarning: invalid escape sequence '\d'
<>:123: SyntaxWarning: invalid escape sequence '\d'
<>:226: SyntaxWarning: invalid escape sequence '\d'
<>:234: SyntaxWarning: invalid escape sequence '\d'
<>:47: SyntaxWarning: invalid escape sequence '\d'
<>:59: SyntaxWarning: invalid escape sequence '\d'
<>:123: SyntaxWarning: invalid escape sequence '\d'
<>:226: SyntaxWarning: invalid escape sequence '\d'
<>:234: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Samiksha Shinde\AppData\Local\Temp\ipykernel_53376\558271627.py:47: SyntaxWarning: invalid escape sequence '\d'
  publish_date = re.findall('\d+/\d+/\d{4} \d+:\d+ \w+',publish_date)[0]
C:\Users\Samiksha Shinde\AppData\Local\Temp\ipykernel_53376\558271627.py:59: SyntaxWarning: invalid escape sequence '\d'
  est_amount = re.sub("[^\d\.\,]", "", est_amount)
C:\Users\Samiksha Shinde\AppData\Local\Temp\ipykernel_53376\558271627.py:123: SyntaxWarning: invalid escape sequence '

https://community.secop.gov.co/Public/Tendering/ContractNoticeManagement/Index?currentLanguage=es-CO&Page=login&Country=CO&SkinName=CCE
audio_src:: https://www.google.com/recaptcha/api2/payload?p=06AFcWeA6cueByhko66PilrJ2YG02n9B7cDT50ldTPtxMs-tDO-n8PpskbI4_fJDSUJqYSwS01q00WS48xAyNz0fUc1IPFCwEcqrtr_n_AW_-Fb032noVXGFFKJIqgb_Ky3WeeFzpEKygIE0v4jFWwAc2p5ooMtzEo23U7CCb-MGSw_ONsczThV47Xo0ARqiFQgpNFyab8kPAMuxPmY7mNDA529_pLVotJNA&k=6LcMmakZAAAAAB157Q90hORUGtNd790TCws4vBNw
Adjusting for ambient noise...
Listening to the audio...
Recognizing speech...
Recognized Text: the fastest drives
Multiple correct solutions required - please solve more.
audio_src:: https://www.google.com/recaptcha/api2/payload?p=06AFcWeA4NbxuzNIZEist2jRN2FevJok8D_r5phZl2IF9X6xdo_4UhODSFRqS3vkhZPoZMXtSzoGjABqCOijrbfpiKBjmx38Sx0RGgpo296b0fAWPMgF0NraJwRoQ7KiTwVUSPSRC4-X7tdc9O2REzIsqunjO-Q-iWle6BhqZvgQzIlIB3Z8sq9piMnu2cTn7OlAFyY4zi_DVB&k=6LcMmakZAAAAAB157Q90hORUGtNd790TCws4vBNw
Adjusting for ambient noise...
Listening to the au

In [2]:
#pip install SpeechRecognition

In [3]:
#pip install pydub

In [4]:
#pip install google-generativeai

In [5]:
#pip install subprocess.run